# **Modelo LightGBM**

Este modelo de xgboost te permite hacer los entrenamientos y predicciones del siguiente mes

**Importar Librerías**

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as ltb
from xgboost import plot_importance, plot_tree
from sklearn.metrics import mean_squared_error, mean_absolute_error
from pandas.tseries.offsets import DateOffset

**Importar Datos**

In [2]:
df = pd.read_csv('dataset_hoteles_ve.csv')

#Crear Modelo
df['date'] = pd.to_datetime(df['date'])
df['date_int'] = df['date'].dt.strftime('%m%d')
df['date_int'] = df['date_int'].astype(np.int32)

#Setear Indice
df = df.set_index('date')
df.index = pd.to_datetime(df.index)
df

C:\Users\felix\AppData\Local\Temp\ipykernel_14832\4067196517.py:1: DtypeWarning: Columns (17,18,20,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dataset_hoteles_ve.csv')


,Unnamed: 0,venue_code,found,rooms_available,rooms_occupied,room_revenue,state,hotelSearchesMX,is_holiday,hocc,...,occ,Evento,Lugar,Modalidad,State,Lugar_Encode,incode,Centro,Distancia Km,date_int
date,,,,,,,,,,,,,,,,,,,,,
2019-02-01,0,BJXCR,True,155,49,77980.96,Guanajuato,0.79,0,0.0000,...,0.316129,0,0,0.0,0,3,0,0,0.0,201
2019-02-02,1,BJXCR,True,155,135,241804.51,Guanajuato,0.98,0,0.0000,...,0.870968,0,0,0.0,0,3,0,0,0.0,202
2019-02-03,2,BJXCR,True,155,102,178099.06,Guanajuato,1.00,0,0.0000,...,0.658065,0,0,0.0,0,3,0,0,0.0,203
2019-02-04,3,BJXCR,True,155,36,51425.74,Guanajuato,0.96,1,0.0000,...,0.232258,0,0,0.0,0,3,0,0,0.0,204
2019-02-05,4,BJXCR,True,155,40,54927.35,Guanajuato,0.86,1,0.0000,...,0.258065,0,0,0.0,0,3,0,0,0.0,205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-26,36998,TRCHA,True,151,54,57313.47,Coahuila,0.77,0,0.0145,...,0.357616,0,0,0.0,0,3,0,0,0.0,626
2022-06-27,36999,TRCHA,True,151,77,80987.93,Coahuila,0.82,0,0.0074,...,0.509934,0,0,0.0,0,3,0,0,0.0,627
2022-06-28,37000,TRCHA,True,151,86,94410.85,Coahuila,0.84,0,0.0238,...,0.569536,0,0,0.0,0,3,0,0,0.0,628


**Dividir Dataset**

In [3]:
#Hacemos el split
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)

variables = ['date_int','hocc','rooms_available','venue_encode','dayofweek','day','month','hotelSearches','hotelSearchesMX','Lugar_Encode','Modalidad']

#Obtenermos las variables independientes y el target
x_train = train[variables].values
y_train = train[['occ']].values

x_test = test[variables].values
y_test = test[['occ']].values

Se hace feature scaling

In [4]:
#Se hace feature scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)
scaler.transform(x_train)

scaler = StandardScaler()
scaler.fit(x_test)
scaler.transform(x_test)

array([[ 1.61581611, -0.90959312,  0.63502011, ...,  0.07416961,
         0.1430468 , -0.13527525],
       [-1.22352769, -0.90959312,  0.59390825, ...,  0.07416961,
         0.1430468 , -0.13527525],
       [ 0.8178877 , -0.90959312, -1.11223369, ..., -1.0293067 ,
         0.1430468 , -0.13527525],
       ...,
       [ 0.51211221, -0.90959312, -0.82445071, ...,  0.97701387,
         0.1430468 , -0.13527525],
       [-0.66439537,  0.81252359,  0.22390157, ...,  1.47859402,
         0.1430468 , -0.13527525],
       [ 1.31877707,  0.60497207,  0.26501342, ..., -0.92899067,
         0.1430468 , -0.13527525]])

**Crear Modelo**

In [5]:
from sklearn.metrics import r2_score
reg = ltb.LGBMRegressor(n_estimators=500)
reg.fit(x_train, y_train)
print(); print(reg)
r2_score(y_test,reg.predict(x_test))

c:\Users\felix\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



LGBMRegressor(n_estimators=500)


0.7692289538739561

**Parameter Tuning**

In [ ]:
#Hyperparameter tuning
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV

parameters = {
              'loss_function':['RMSE'],
              'learning_rate': [0.01,0.03,0.05,0.1],
              'max_depth': [5,6,7,9,10,12,14,16,18],
              'min_data_in_leaf':[10,20,30,40,50,60,70,80,90,100,110,120,130,150,170,200],
              'bagging_fraction':[0.1,0.2,0.3,0.5,0.7,0.9,1],
              'num_leaves':[10,15,30,35,40,50,60,70,80,90,100],
              'bagging_freq':[0,1,3,5,7,9,10]
              }

lgb_hcv = HalvingGridSearchCV(reg, 
                              parameters, 
                              scoring="r2", 
                              n_jobs=-1, 
                              min_resources="exhaust", 
                              factor=3,
                              cv=3,
                              verbose=1)
lgb_hcv.fit(x_train, y_train)
print(lgb_hcv.best_score_)
print(lgb_hcv.best_params_)

**Reentrenar Modelo**

In [ ]:
lgb_best = ltb.LGBMRegressor()
lgb_best.fit(x_train, y_train)
r2_score(y_test,lgb_best.predict(x_test))

**Guardar y Cargar el Modelo**

In [ ]:
reg.booster_.save_model('LightGBM_SinDistancia.txt')

In [ ]:
loaded_model = ltb.Booster(model_file='LightGBM_SinDistancia.txt')
loaded_model

**Predecir Datos con el Modelo Cargado**

In [ ]:
from pandas.tseries.offsets import DateOffset
def ModelPredictOcc(df,vc):

    #Cargar Modelo
    loaded_model = ltb.Booster(model_file='LightGBM_SinDistancia.txt')

    #Obtener datos del último mes para el venue especificado en el dataset
    df_venue = df[df['venue_code'] == vc]
    lastdayfrom = max(df_venue.index)
    df_venue_last = df_venue.loc[(df_venue.index.month==lastdayfrom.month) & (df_venue.index.year==lastdayfrom.year)]
    df_venue_last = df_venue_last[['date_int','hocc','rooms_available','venue_encode','dayofweek','day','month','hotelSearches','hotelSearchesMX','Lugar_Encode','Modalidad']]
    #features = df_venue_last.copy()

    #Obtener la fecha del dataframe para generar un dataframe esqueleto en donde almacenar las predicciones
    #de las fechas futuras de las que no se tiene registro de datos todavía.
    df_venue_last.reset_index(inplace=True)
    df_predicted = df_venue_last[['date']]

    #Predecir ocupación
    features = df_venue_last.copy()
    del features['date']
    prediction = loaded_model.predict(features.values)
    df_predicted['predicted_occ'] = prediction

    #Truncar datos dependiendo del mes siguiente. Si se tiene un mes siguiente con 30 días o menos se recorta el dataframe
    #Para esto se hace un offset de las fechas asociadas a las predicciones. Se hace de forma automática
    new_dates = []
    for date in df_predicted['date']:
        new_date = date + DateOffset(months=1)
        new_dates.append(new_date)
    df_predicted['date'] = new_dates

    #Reestablecer las fechas en caso de que haya días en los que no se registró la ocupación.
    df_predicted_new = df_predicted.copy()
    len_obs = len(df_predicted)
    df_predicted_new['date'] = pd.date_range(start=min(df_predicted_new['date']), periods=len_obs)
    

    return df_predicted_new

In [ ]:
#Caso de ejemplo de uso de la función
resultados = ModelPredictOcc(df,'BJXCR')
resultados